In [ ]:
import os
import cv2
import shutil

# Function to apply an edge smoothing technique and save the result to a folder
def apply_edge_smoothing(original_folder, output_folder, smoothing_function, **params):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(original_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(original_folder, filename)
            original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Apply the edge smoothing technique
            smoothed_image = smoothing_function(original_image, **params)

            # Save the smoothed image to the output folder
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, smoothed_image)

# Define the paths for the original segmented images folder and output folders for each technique
original_images_folder = "path/to/original_images_folder"
output_gaussian_folder = "path/to/output/gaussian"
output_bilateral_folder = "path/to/output/bilateral"
output_median_folder = "path/to/output/median"
output_anisotropic_folder = "path/to/output/anisotropic"
output_non_local_means_folder = "path/to/output/non_local_means"
output_gradient_based_folder = "path/to/output/gradient_based"

# Apply different edge smoothing techniques to the original images
apply_edge_smoothing(original_images_folder, output_gaussian_folder, gaussian_blur, kernel_size=(5, 5), sigmaX=0)
apply_edge_smoothing(original_images_folder, output_bilateral_folder, bilateral_filter, d=9, sigma_color=75, sigma_space=75)
apply_edge_smoothing(original_images_folder, output_median_folder, median_filter, kernel_size=3)
apply_edge_smoothing(original_images_folder, output_anisotropic_folder, anisotropic_diffusion, n_iter=50, delta_t=0.1, kappa=100)
apply_edge_smoothing(original_images_folder, output_non_local_means_folder, non_local_means, h=10, h_for_color=10, template_window_size=5, search_window_size=21)
apply_edge_smoothing(original_images_folder, output_gradient_based_folder, gradient_inverse_weighted_interpolation, alpha=0.2, threshold=30)
